In [0]:

import lakefs_client
from lakefs_client import models
from lakefs_client.client import LakeFSClient

# lakeFS credentials and endpoint
configuration = lakefs_client.Configuration()
configuration.username = '{your user name}'
configuration.password = '{your password}'
configuration.host = 'https://{some-aws-host}.lakefs.io'


In [0]:

client = LakeFSClient(configuration)

In [0]:
%python
client.branches.list_branches('dais22-adi')

Out[203]: {'pagination': {'has_more': False,
 'max_per_page': 1000,
 'next_offset': '',
 'results': 1},
 'results': [{'commit_id': 'fcd0d0abe348b19e0c33ceb1b6b74a8e3902a9483fbf5aa4642e4f49eee37d68',
 'id': 'main'}]}

Create new branch

In [0]:

client.branches.create_branch(repository='dais22-adi', branch_creation=models.BranchCreation(name='experiment-chaos', source='main'))

Out[204]: 'fcd0d0abe348b19e0c33ceb1b6b74a8e3902a9483fbf5aa4642e4f49eee37d68'

In [0]:

main_repo_path = "lakefs://dais22-adi/main/"
chaos_repo_path = "lakefs://dais22-adi/experiment-chaos/"

## Diffing a single branch will show all uncommitted changes on that branch:

In [0]:

client.branches.diff_branch(repository='dais22-adi', branch='experiment-chaos').results

Out[205]: []

In [0]:

genre_df = spark.read.format("parquet").load(chaos_repo_path+"genres")

In [0]:
%python
books_df = spark.read.format("parquet").load(chaos_repo_path+"books")

In [0]:

books_df.show(10, truncate=False)

+--------+-------------------------------------------------------------+-------------------+
isbn |name |author |
+--------+-------------------------------------------------------------+-------------------+
32278888|Hello Spark Fans |Advanced Analytics |
73825104|Fundamentals of Data Observability |Andy Petrella |
73825103|High Performance Spark |Holden Karau |
73341143|Data Engineering with Apache Spark, Delta Lake, and Lakehouse|Manoj Kukreja |
54725104|Fundamentals of Data Observability |Andy Petrella |
54725222|Designing Data-Intensive Applications |Martin Kleppmann |
54725283|Data Management at Scale |Piethein Strengholt|
29829283|Database Internals |Alex Petrov |
25678345|Scaling Data Platforms |Gamora |
39898782|Project metadata |catwoman |
+--------+-------------------------------------------------------------+-------------------+
only showing top 10 rows

In [0]:

genre_df.show(10, truncate=False)

+--------+-------------------------------------------------------------+---------+
isbn |name |genre |
+--------+-------------------------------------------------------------+---------+
32278888|Hello Spark Fans |classics |
73825104|Fundamentals of Data Observability |adventure|
73341143|Data Engineering with Apache Spark, Delta Lake, and Lakehouse|adventure|
54725104|Fundamentals of Data Observability |classics |
42278345|Intro to Hive metastore |mystery |
52278888|Reviving zookeper |drama |
62278888|Life after Hadoop |crime |
83825104|Fundamentals of Lakehouse |adventure|
93825104|High Performance Yarn |fiction |
54725222|Designing Data-Intensive Applications |classics |
+--------+-------------------------------------------------------------+---------+
only showing top 10 rows

In [0]:
%sql
DROP TABLE IF EXISTS books

In [0]:

books_df.write.saveAsTable("books")

In [0]:
%sql
DROP TABLE IF EXISTS genre

In [0]:

genre_df.write.saveAsTable("genre")

## Join operation

In [0]:

data = genre_df.join( books_df, genre_df.isbn ==  books_df.isbn, "left" ).select(books_df.isbn, books_df.name, books_df.author, genre_df.genre)

## Save the materialized view

In [0]:

data.write.mode('overwrite').parquet(chaos_repo_path+"books-dataset")

In [0]:

data.show(20,truncate=False)

+--------+-------------------------------------------------------------+-------------------+---------+
isbn |name |author |genre |
+--------+-------------------------------------------------------------+-------------------+---------+
32278888|Hello Spark Fans |Advanced Analytics |classics |
73825104|Fundamentals of Data Observability |Andy Petrella |adventure|
73341143|Data Engineering with Apache Spark, Delta Lake, and Lakehouse|Manoj Kukreja |adventure|
54725104|Fundamentals of Data Observability |Andy Petrella |classics |
42278345|Intro to Hive metastore |she-hulk |mystery |
52278888|Reviving zookeper |dr-strange |drama |
62278888|Life after Hadoop |Green Arrow |crime |
83825104|Fundamentals of Lakehouse |Barry Allen |adventure|
93825104|High Performance Yarn |Harley Quinn |fiction |
54725222|Designing Data-Intensive Applications |Martin Kleppmann |classics |
54725283|Data Management at Scale |Piethein Strengholt|classics |
25678345|Scaling Data Platforms |Gamora |drama |
39898782|Project metadata |catwoman |adventure|
null |null |null |fiction |
15678345|Building Data Platforms |Gamora |drama |
89898782|Scaling metadata |catwoman |mystery |
32278345|Beyond the clean data curve |batman |tragedy |
+--------+-------------------------------------------------------------+-------------------+---------+

## Run Quality checks on the experimentation brach

In [0]:

from pyspark.sql.functions import col,isnan, when, count
data.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]
   ).show()

+----+----+-----+
isbn|name|genre|
+----+----+-----+
 1| 1| 0|
+----+----+-----+

In [0]:

data.count()

Out[215]: 17

In [0]:

genre_df.count()

Out[216]: 17

In [0]:

books_df.count()

Out[217]: 20

In [0]:

display(genre_df)

isbn,name,genre
32278888,Hello Spark Fans,classics
73825104,Fundamentals of Data Observability,adventure
73341143,"Data Engineering with Apache Spark, Delta Lake, and Lakehouse",adventure
54725104,Fundamentals of Data Observability,classics
42278345,Intro to Hive metastore,mystery
52278888,Reviving zookeper,drama
62278888,Life after Hadoop,crime
83825104,Fundamentals of Lakehouse,adventure
93825104,High Performance Yarn,fiction
54725222,Designing Data-Intensive Applications,classics


## join operation #2nd try

In [0]:

data_v2 = books_df.join( genre_df, genre_df.isbn ==  books_df.isbn, "left" ).select(books_df.isbn, books_df.name, books_df.author, genre_df.genre)

In [0]:

from pyspark.sql.functions import col,isnan, when, count
data_v2.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]
   ).show()

+----+----+-----+
isbn|name|genre|
+----+----+-----+
 0| 0| 4|
+----+----+-----+

In [0]:
%python
genre_df.count()

Out[221]: 17

In [0]:

books_df.count()

Out[130]: 20

In [0]:

data.count()

Out[131]: 17

## fix missed data

In [0]:

data_v3 = data_v2.fillna("classics",subset=["genre"])
data.write.mode('overwrite').parquet(chaos_repo_path+"books-dataset")

In [0]:

client.branches.diff_branch(repository='dais22-adi', branch='experiment-chaos').results

Out[223]: [{'path': 'books-dataset/_committed_9141075280486096233',
 'path_type': 'object',
 'size_bytes': 1314,
 'type': 'added'},
 {'path': 'books-dataset/_started_9141075280486096233',
 'path_type': 'object',
 'size_bytes': 1314,
 'type': 'added'},
 {'path': 'books-dataset/part-00000-tid-4105526429050582069-ae3cb776-571d-4b93-a816-7c52b0e346f7-1442-1-c000.snappy.parquet',
 'path_type': 'object',
 'size_bytes': 1314,
 'type': 'removed'},
 {'path': 'books-dataset/part-00000-tid-9141075280486096233-63adee4b-f45b-4fbc-a216-00c273de0215-1509-1-c000.snappy.parquet',
 'path_type': 'object',
 'size_bytes': 1314,
 'type': 'added'},
 {'path': 'books-dataset/part-00001-tid-4105526429050582069-ae3cb776-571d-4b93-a816-7c52b0e346f7-1443-1-c000.snappy.parquet',
 'path_type': 'object',
 'size_bytes': 1314,
 'type': 'removed'},
 {'path': 'books-dataset/part-00001-tid-9141075280486096233-63adee4b-f45b-4fbc-a216-00c273de0215-1510-1-c000.snappy.parquet',
 'path_type': 'object',
 'size_bytes': 1314,
 'type': 'added'},
 {'path': 'books-dataset/part-00002-tid-4105526429050582069-ae3cb776-571d-4b93-a816-7c52b0e346f7-1444-1-c000.snappy.parquet',
 'path_type': 'object',
 'size_bytes': 1314,
 'type': 'removed'},
 {'path': 'books-dataset/part-00002-tid-9141075280486096233-63adee4b-f45b-4fbc-a216-00c273de0215-1511-1-c000.snappy.parquet',
 'path_type': 'object',
 'size_bytes': 1314,
 'type': 'added'},
 {'path': 'books-dataset/part-00003-tid-4105526429050582069-ae3cb776-571d-4b93-a816-7c52b0e346f7-1445-1-c000.snappy.parquet',
 'path_type': 'object',
 'size_bytes': 1314,
 'type': 'removed'},
 {'path': 'books-dataset/part-00003-tid-9141075280486096233-63adee4b-f45b-4fbc-a216-00c273de0215-1512-1-c000.snappy.parquet',
 'path_type': 'object',
 'size_bytes': 1314,
 'type': 'added'}]

# Git like interface - Branching out

![](https://docs.lakefs.io/assets/img/branching_7.png)
<!-- ![](https://miro.medium.com/max/1400/0*2N9qc0DlQmD_nK_Q.png) -->
<!-- <img src="https://miro.medium.com/max/1400/0*2N9qc0DlQmD_nK_Q.png" alt="drawing" style="width:50px;height:20px;"/> -->

## Cross collection consistency
We often need consistency between different data collections. A few examples may be:

* To join different collections in order to create a unified view of an account, a user or another entity we measure.
* To introduce the same data in different formats
* To introduce the same data with a different leading index or sorting due to performance considerations

![](https://docs.lakefs.io/assets/img/branching_8.png)

## We need to modify the materialized view!

In [0]:
%python
data_v2 = books_df.join( genre_df, genre_df.isbn ==  books_df.isbn, "left" ).select(books_df.isbn, books_df.name, books_df.author, genre_df.genre)

In [0]:

from pyspark.sql.functions import col,isnan, when, count
data_v2.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]
   ).show()

+----+----+-----+
isbn|name|genre|
+----+----+-----+
 0| 0| 4|
+----+----+-----+

In [0]:

data.write.mode('overwrite').parquet(chaos_repo_path+"books-dataset")

In [0]:

data_v3 =  data_v2.fillna("classics", subset=["genre"])